<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_answer_BERT_nsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질문에 대한 답변을 선별하는 코드(BERT Next Sentence Prediction)(로컬)

In [ ]:
!pip install datasets
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#모델 불러오기
HUGGINGFACE_MODEL_PATH = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = BertForNextSentencePrediction.from_pretrained(HUGGINGFACE_MODEL_PATH)

In [ ]:
#모델 실험
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors="pt")

outputs = model(**encoding, labels=torch.LongTensor([1]))
logits = outputs.logits
assert logits[0, 0] < logits[0, 1]  # next sentence was random

{'sequence': 'Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU', 'labels': ['politics', 'economy', 'environment', 'entertainment'], 'scores': [0.9658799171447754, 0.022846776992082596, 0.007333964109420776, 0.003939350135624409]}
politics


In [ ]:
#훈련 데이터 불러오기
dataset_path = '/content/drive/My Drive/community_train_dataset_result.csv'

In [ ]:
#카카오톡 데이터 불러오기
file_path = '/content/drive/My Drive/judge_question_result.csv'

In [ ]:
#카카오톡 대화내용을 데이터프레임으로 받기
df = pd.read_csv(file_path)

#질문 딕셔너리, 답변 목록 리스트(이중 리스트) 생성
questions, answer_lists = {}, []

#질문으로 판별된 텍스트를 새 데이터프레임으로 생성
df_question = df[df['label'] == 'question']

print(df_question)
#새 데이터프레임의 index, text를 question 딕셔너리에 저장
questions = {text : index for (index, text) in zip(df_question.index, df_question['text'])}
print(questions)

      Unnamed: 0.1  Unnamed: 0            name  \
0                0           0   조용찬 / 백엔드 개발자   
4                4           4        조한울 / 운영   
7                7           7              제제   
19              19          19   정용준 / Litmers   
35              35          36        김호종/스타트업   
...            ...         ...             ...   
7935          7935        7960   하헌준 / 백엔드 개발자   
7936          7936        7961             김승진   
7941          7941        7966          퐈이팅해야지   
7944          7944        7969        버블챌린지하실분   
7948          7948        7973             김승진   

                                                   text     label  
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.\ndelt...  question  
4      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한...  question  
7                         Bubble supportor에 문의 해 보셨어요??  question  
19                   아이고.. 워크로드가 과중될것으로 예상되는 부분이 있으신가요?  question  
35                                10번 할거 1번으로 어떻게 끝내나요?  qu

In [ ]:
# Ensure the classifier is using GPU
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU
print(device)
classifier = pipeline('zero-shot-classification', model=HUGGINGFACE_MODEL_PATH, device=device, batch_size = 64)

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-multilingual-cased')

# 문장 정의
sentence_a = "안녕하세요. 저는 학생입니다."
sentence_b = "공부를 많이 하고 있어요."

# 인코딩 및 NSP 태스크 준비
encoded = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt')
with torch.no_grad():
    outputs = model(**encoded, next_sentence_label=torch.LongTensor([1]))
    logits = outputs.logits

# 결과 해석
predicted_label = torch.argmax(logits).item()

if predicted_label == 1:
    print("문장 B는 문장 A의 뒤에 올 수 있습니다.")
else:
    print("문장 B는 문장 A의 뒤에 올 수 없습니다.")


In [ ]:
#전체 질문에 대해서 각 질문 이후 30개의 텍스트에 대해서 판별, 이때 특정 텍스트가 답변이 맞으면 답변 리스트에 추가한 뒤 다음 텍스트 판별

# Pre-fetch the necessary DataFrame data to minimize access within loops
question_texts = {question: df.iloc[questions[question] + 1 : questions[question] + 6]['text'].tolist() for question in questions.keys()}

answer_lists = []
i = 0
for question, texts in tqdm(question_texts.items(), desc="Processing questions"):
    answer = []
    for text in texts: #나중에 숫자 커지면 texts 대신 tqdm(texts, desc="Classifying texts", leave=False)
        sequence_to_classify = question + ' '.join(answer + [text])
        #동적 레이블
        candidate_labels = ["continuing answer", "not continuing answer"] if answer else ["question-answer pair", "not question-answer pair"]

        # Process the classification in one step
        output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
        expected_label = "continuing answer" if answer else "question-answer pair"
        if output['labels'][0] == expected_label:
            answer.append(text)
    answer_lists.append(answer) if len(answer) != 0 else answer_lists.append(['No answer'])

In [ ]:
print(answer_lists, end = '\n')

[[' [IT 종사자들의 비즈니스 영어 회화 모집!]\n\nIT 업계에서 일하시는 분들과 함께 영어 회화 모임을 만들어서 꾸준히 영어 공부도 하고 네트워킹도 구축할 수 있는 모임을 만드는 중인데요!\n\n오프라인 영어 회화 모임을 나가기에는 번거로웠던 분들을 위해 온라인으로 진행할 예정입니다! 매일 저녁 6시부터 12시까지 원하는 시간에 자유롭게 들어오실 수 있습니다!\n\n비슷한 수준의 영어 실력인 분들과 그룹을 만들어드리니 실력과 영어 회화 수준에 대해서는 크게 걱정하지 않으셔도 됩니다! \n\n모임에 조금이라도 관심이 있으신 분은 오늘 저녁 10시 줌에서 커뮤니티 소개 세션을 진행하려고 합니다!\n\n아래 링크를 통해서 신청해주시면 따로 연락 드려서 안내해드릴게요! 많은 관심 부탁드립니다! :)', ' 유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분이 있어 질문드립니다!', ' 혹시 버블 워크로드 최적화하는 \x1c팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데, 곧 요금제를 업그레이드 안 하면 감당이 안 될 거 같은데요.\n계산해보니 예상 요금이 월 400만원이 나오게 생겼네요..ㅠㅠ', ' 으헉 ..\n 백엔드만 사용하셔도 그렇게 많이 나오는군요 ..'], [' Bubble supportor에 문의 해 보셨어요??', ' 와..ㄷ ㄷ ㄷ', ' 이정도면 세일즈랑 논의가 필요해 보여요'], [' 와..ㄷ ㄷ ㄷ', ' 이정도면 세일즈랑 논의가 필요해 보여요'], [' 저희 유저 분들이 댓글도 엄청 다시고 좋아요도 엄청 누르시고 하셔서 그 부분에서 워크로드를 많이 쓰고 있긴 합니다..'], [' 리피팅 그룹 or 팝업그룹 or 그룹에 do a search for로 초기 데이터를 불러 오고 다른 엘리먼트 이 그룹의 데이터를 가져오는 걸로 하는 겁니다.'], [' 가능할 것 같은데용?', ' 요런 알람은 어디에서 수정할 수 있나요?', ' Setting>Language에 있습니다', ' 카페24 api 와 연동해 보신분계

In [ ]:
# #데이터프레임 가공
# df_question.rename(columns = {'text' : 'question'})
# df_question['answer'] = answer_lists

data = {question : answer for question, answer in zip(questions.keys(), answer_lists)}
print(data)
data_result= [{'question' : key, 'answer' : value} for key, value in data.items()]
df_result = pd.DataFrame(data = data_result, columns = ['question', 'answer'])
print(df_result)

In [ ]:
#.csv 파일로 google drive에 저장
save_path = '/content/drive/My Drive/judge_answer_result.csv'

df_result.to_csv(save_path)